In [ ]:
"""
Purpose of this table:
1) For all of the postysn segments WITH SOMA (excluding orphans)
computes the n_shared_segments and the n_shared_synapses as well as the unions
"""

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import itertools 
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import datajoint as dj
from collections import defaultdict
import pycircstat as pycs
from funconnect import morphology, connectomics, ta3, nda
from tqdm import tqdm
import scipy.stats as stats


Connecting celiib@10.28.0.34:3306


In [3]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20


pinky = dj.create_virtual_module('pinky', 'microns_pinky')
pinky_nda = dj.create_virtual_module('pinky_nda', 'microns_pinky_nda')
pinky_radtune = dj.create_virtual_module('pinky_radtune', 'microns_pinky_radtune')
pinky_spattune = dj.create_virtual_module('pinky_spattune', 'microns_pinky_spattune')
pinky_fc = dj.create_virtual_module('pinky_fc', 'microns_pinky_fc')


In [24]:
pinky_radtune.VonFit()
pinky_radtune.VonHash()
pinky_radtune.BestVonFit() #only has the vonbestfitmethod
pinky_radtune.BestVonFit.Unit() #has specific numbers for each segment_id
pinky_radtune.VonCorr() #has the pair-pair differences
pinky_radtune.RadHash() #empty
pinky_radtune.BestRadFit() #empty
pinky_radtune.BestRadFit.Unit() #empty
pinky_radtune.VonFit.Unit()
pinky_radtune.RadTempCorr()

radtemp_hash1 unique hash for combination of sta analysis methods,radtemp_hash2 unique hash for combination of sta analysis methods,rtt_rad_corr pearson corr btw radtemp fit radial components,rtt_rad_corr_pvalue pearson corr pvalue btw radtemp fit radial components,rtt_temp_corr pearson corr btw radtemp fit temporal components,rtt_temp_corr_pvalue pearson corr pvalue btw radtemp fit temporal components,rtt_full_corr pearson corr btw radtemp full fitted matrix,rtt_full_corr_pvalue pearson corr pvalue btw radtemp full fitted matrix,diff_rtt_adv difference in unshuffled radtemp predictive advantage,diff_rtt_pvalue difference in unshuffled radtemp p value


In [5]:
from utils import find_pearson, find_cosine, find_binary_sim

schema = dj.schema('microns_pinky_fc', create_tables=True)


In [8]:
pinky.AllenSoma()

segmentation segmentation id,segment_id segment id unique within each Segmentation,soma_id soma id in Allen annotation database,soma_x x location of soma in grid coordinates,soma_y y location of soma in grid coordinates,soma_z z location of soma in grid coordinates,cell_class cell class
3,648518346341371119,329,105998,71204,669,excitatory
3,648518346341373204,417,59596,69165,2132,glia
3,648518346341380687,433,53383,62029,354,glia
3,648518346341382424,415,58087,60834,159,glia
3,648518346341388647,424,80108,56958,2080,glia
3,648518346341394877,458,94337,41289,2133,glia
3,648518346342018870,445,79431,67616,403,glia
3,648518346342162152,455,69091,74601,1902,glia
3,648518346342792054,423,116198,61856,2148,glia
3,648518346342792054,454,114059,59673,1975,glia


In [9]:
pinky_nda.All

AttributeError: module 'pinky_nda' has no attribute 'AllenCellClass'

In [10]:
@schema
class PostSynapticPairInput(dj.Computed):
    definition = """
    -> pinky.Segment
    segment_b           : bigint   # segment id unique within each Segmentation
    ---
    n_syn_a     : int # number of synapses for cell A
    n_syn_b     : int # number of synapses for cell B
    n_syn_union : int # number of unique synapses for both
    n_syn_shared: int # number of shared synapses
    n_seg_a     : int # number of segments for cell A
    n_seg_b     : int # number of segments for cell B
    n_seg_union : int # number of unique segments for both
    n_seg_shared: int # number of shared segments
    """

    @property
    def key_source(self):
        return pinky.Segmentation & pinky.CurrentSegmentation

    def make(self, key):
        soma = pinky.Segment & pinky.AllenSoma & (pinky.AllenSoma & 'cell_class="excitatory"') #pinky_nda.Trace & key
        axons = pinky.Neurite & 'neurite_type="axon"'
        synapse = pinky.Synapse & axons.proj(presyn='segment_id') & key
        info = soma * soma.proj(segment_b='segment_id')

        A = (synapse & soma.proj(postsyn='segment_id')).proj(
            'presyn', syn1='synapse_id', segment_id='postsyn')
        B = (synapse * dj.U('presyn') & soma.proj(postsyn='segment_id')).proj(
            'presyn', syn2='synapse_id', segment_b='postsyn')
        shared = dj.U('segment_id', 'segment_b').aggr(A * B & 'segment_id > segment_b',
                                                      n_syn_shared='count(*)',
                                                      n_seg_shared='count(DISTINCT presyn)')
        a = dj.U('segment_id').aggr(A, n_syn_a='count(*)', n_seg_a='count(DISTINCT presyn)')
        b = dj.U('segment_b').aggr(B, n_syn_b='count(*)', n_seg_b='count(DISTINCT presyn)')
        stats = (info * a * b * shared).proj('n_syn_a', 'n_syn_b', 'n_syn_shared',
                                             'n_seg_a', 'n_seg_b', 'n_seg_shared',
                                             n_syn_union='n_syn_a + n_syn_b - n_syn_shared',
                                             n_seg_union='n_seg_a + n_seg_b - n_seg_shared',
                                             )
        self.insert(stats, ignore_extra_fields=True)

In [11]:
import time
start_time = time.time()
PostSynapticPairInput.populate()
print(f"Total time = " + str(time.time() - start_time))



Total time = 46.213337898254395
